In [ ]:
'''This script contains all the initialization code required in testing and calibration.

It includes:
1) Yield Curve construction
2) Black and Chi Square Pricers
3) Hagan Approximations for CEV and Shifted CEV

All the necessary packages have also been imported. 

All reference material guiding the code has been provided in the Documentation section of the Economic Scenario
Generator folder.'''

In [30]:
%%capture
%pylab inline --no-import-all
import numpy as np
import pandas as pd
import os
from math import *

# Computational Packages
from scipy import stats, optimize
from QuantLib import *
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve, calibrate


# HTML Display
import base64 
from IPython.display import HTML, display

# Looping Packages
import itertools
import tabulate
import zipfile


# Plotting Functions
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib as mpl
import matplotlib.pyplot as plt
import utils

# Timers and Date Wranglers
import time
from timeit import timeit
from datetime import datetime

# Parallel Processing
from numba import jit, njit, prange, autojit

In [22]:
'''OBTAIN YIELD CURVE DATA (EONIA RATES)'''
'''These rates will be used for OIS Discounting in swaption pricing'''
loc = "Data\Calibration Data.xlsx"

# Obtain OIS Discount Data
df = pd.read_excel(loc, 
                   sheet_name = "EONIA Rates",
                   skiprows= [0, 1,3],
                   index_col = "Dates")

df = pd.DataFrame.dropna(df, axis = 'rows')

#Obtain ATM Black Volatility Data
vol_data = pd.read_excel(loc,
                        sheet_name = "ATM Swaptions Black Vol",
                        skiprows= [0, 1,3,4],
                        index_col = "Dates")

vol_data = pd.DataFrame.dropna(vol_data, axis = 'rows')

# Obtain Swap Curve Data
swap_data = pd.read_excel(loc,
                        sheet_name = "Swap Rates Data",
                        skiprows= [0, 1,3],
                       index_col = "Dates")

swap_data = pd.DataFrame.dropna(swap_data, axis = 'rows')

In [29]:
def swap_curve_construct(date):
    e_date = DateParser.parseFormatted(date, '%Y-%m-%d')
    Settings.instance().setEvaluationDate(e_date)
    
    Swap_Mat = ([Period(i, Years) for i in range(1, 31)]+
                    [Period(i*5, Years) for i in range(7, 13)])

    #Select the date's Curve
    Swap_Mat_Rates = swap_data.loc[date]

    ## Swap Rate Helpers
    '''We create rate helpers based on the definition of the OIS contract on Bloomberg'''
    swap_helpers = [SwapRateHelper(QuoteHandle(SimpleQuote(rate/100)), # Swap Rate
                              tenor, 
                              TARGET(),
                              Annual, #Fixed Leg Frequency
                              Unadjusted, #Fixed leg adjustment
                              Thirty360(), #Day Counter
                              Euribor6M()) #Floating Rate 
               for rate, tenor in zip(Swap_Mat_Rates, Swap_Mat)]

    # Construct the Curve 
    swap_curve_ld = PiecewiseLogLinearDiscount(0, 
                                TARGET(),
                                swap_helpers, 
                                Actual365Fixed())
    swap_curve_ld.enableExtrapolation()

    return(swap_curve_ld)

In [2]:
def eonia_curve_construct(date):
    e_date = DateParser.parseFormatted(date, '%Y-%m-%d')
    Settings.instance().setEvaluationDate(e_date)
    
    SE_Mat = ([Period(1, Weeks), 
              Period(2, Weeks), 
              Period(3, Weeks), 
              Period(4, Weeks)] + 
                [Period(i, Months) for i in range(2, 24)] +  
                [Period(2, Years), Period(30, Months)] +
                [Period(i, Years) for i in range(3, 13)]+
                [Period(i*5, Years) for i in range(3,9)]+
                [Period(50, Years)])

    #date = datetime.strftime(datetime(2019, 5, 2), "%Y-%m-%d")
    SE_Mat_Rates = df.loc[self.date]

    ## EONIA Rate Helpers
    '''We create rate helpers based on the definition of the OIS contract on Bloomberg'''
    helpers = [OISRateHelper(2, #Settlement Days 
                  tenor, #Tenor of the OIS 
                  QuoteHandle(SimpleQuote(rate/100)), #OIS Rate
                  Eonia())
    for rate, tenor in zip(SE_Mat_Rates, SE_Mat)]

    ## Construct EONIA Curve
    eonia_curve_ld = PiecewiseLogLinearDiscount(0, 
                                TARGET(),
                                helpers, 
                                Actual365Fixed())
    eonia_curve_ld.enableExtrapolation()

    return(eonia_curve_ld)

In [152]:
class model_definition:
    def __init__(self, date):
        self.date = date
    
    def eval_date(self):
        # Set evaluation date
        e_date = DateParser.parseFormatted(self.date, '%Y-%m-%d')
        Settings.instance().setEvaluationDate(e_date)
        
        
    def eonia_construct(self):
        #Set global settings
        self.eval_date()
        
        '''CONSTRUCTION OF THE CURVES'''
        #Input market quotes
        SE_Mat = ([Period(1, Weeks), 
                  Period(2, Weeks), 
                  Period(3, Weeks), 
                  Period(4, Weeks)] + 
                    [Period(i, Months) for i in range(2, 24)] +  
                    [Period(2, Years), Period(30, Months)] +
                    [Period(i, Years) for i in range(3, 13)]+
                    [Period(i*5, Years) for i in range(3,9)]+
                    [Period(50, Years)])

        #date = datetime.strftime(datetime(2019, 5, 2), "%Y-%m-%d")
        SE_Mat_Rates = df.loc[self.date]

        ## EONIA Rate Helpers
        '''We create rate helpers based on the definition of the OIS contract on Bloomberg'''
        helpers = [OISRateHelper(2, #Settlement Days 
                      tenor, #Tenor of the OIS 
                      QuoteHandle(SimpleQuote(rate/100)), #OIS Rate
                      Eonia())
        for rate, tenor in zip(SE_Mat_Rates, SE_Mat)]

        ## Construct EONIA Curve
        eonia_curve_ld = PiecewiseLogLinearDiscount(0, 
                                    TARGET(),
                                    helpers, 
                                    Actual365Fixed())
        eonia_curve_ld.enableExtrapolation()
        
        return(eonia_curve_ld)
    
        '''===================================================================================================================='''
    def swap_construct(self):
        #Set global settings
        self.eval_date()
        
        '''CONSTRUCTION OF THE SWAP DISCOUNT CURVE'''
        # Input market quotes
        Swap_Mat = ([Period(i, Years) for i in range(1, 31)]+
                    [Period(i*5, Years) for i in range(7, 13)])

        #Select the date's Curve
        Swap_Mat_Rates = swap_data.loc[self.date]

        ## Swap Rate Helpers
        '''We create rate helpers based on the definition of the OIS contract on Bloomberg'''
        swap_helpers = [SwapRateHelper(QuoteHandle(SimpleQuote(rate/100)), # Swap Rate
                                  tenor, 
                                  TARGET(),
                                  Annual, #Fixed Leg Frequency
                                  Unadjusted, #Fixed leg adjustment
                                  Thirty360(), #Day Counter
                                  Euribor6M()) #Floating Rate 
                   for rate, tenor in zip(Swap_Mat_Rates, Swap_Mat)]

        # Construct the Curve
        global swap_curve_ld 
        swap_curve_ld = PiecewiseLogLinearDiscount(0, 
                                    TARGET(),
                                    swap_helpers, 
                                    Actual365Fixed())
        swap_curve_ld.enableExtrapolation()

        return(swap_curve_ld)
    
    def payer_Black(self, N, S0, expiry, tenor, K, sigma):
        #Set global settings
        self.eval_date()
        
        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        # Find the schedule of payments for the underlying swap 
        schedule = Schedule(expiry,
             tenor,
             Period(Annual),
             TARGET(),
             ModifiedFollowing,
             ModifiedFollowing,
             DateGeneration.Forward,
             False)

        # Sum of Zero Coupon Bonds
        PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])

        #Define Parameters
        T = Actual365Fixed().yearFraction(eval_date, expiry)
        d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
        d2 = d1 - sigma*sqrt(T)

        #Calculate Price
        price = N*PVBP * (S0*stats.norm.cdf(d1) - K*stats.norm.cdf(d2))
        return(price)

    def receiver_Black(self, N, S0, expiry, tenor, payments, K, sigma):
        #Set global settings
        self.eval_date()
        
        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        # Find the schedule of payments for the underlying swap 
        schedule = Schedule(expiry,
             tenor,
             Period(Annual),
             TARGET(),
             ModifiedFollowing,
             ModifiedFollowing,
             DateGeneration.Forward,
             False)
        
        PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])
        T = Actual365Fixed().yearFraction(eval_date, expiry)
        d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
        d2 = d1 - sigma*sqrt(T)

        price = N * PVBP * (K*stats.norm.cdf(-d2) - S0*stats.norm.cdf(-d1))      
        return(price)    

    def payer_chi_square(self, N, S0, expiry, tenor, K, beta, sigma):
        #Set global settings
        self.eval_date()

        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        schedule = Schedule(expiry,
             tenor,
             Period(Annual),
             TARGET(),
             ModifiedFollowing,
             ModifiedFollowing,
             DateGeneration.Forward,
             False)

        PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])]) 

        # Parameters for input into Chi Square CDF
        d = (pow(K,2 - 2*beta)/(pow(1 - beta,2)*pow(sigma,2)*
                                (Actual365Fixed().yearFraction(eval_date, tenor))))
        b = 1/(1 - beta)
        f = (pow(S0, 2 - 2*beta)/(pow(1 - beta, 2)*pow(sigma, 2)*
                                  (Actual365Fixed().yearFraction(eval_date, tenor))))

        # Calculate Price
        price = N * PVBP * (S0*(1 - stats.ncx2.cdf(d, b+2, f)) - K*(stats.ncx2.cdf(f, b, d)))
        return(price)

    def receiver_chi_square(self, N, S0, expiry, tenor, K, beta, sigma):
        #Set global settings
        self.eval_date()

        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        schedule = Schedule(expiry,
             tenor,
             Period(Annual),
             TARGET(),
             ModifiedFollowing,
             ModifiedFollowing,
             DateGeneration.Forward,
             False)

        PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])

        d = (pow(K,2 - 2*beta)/(pow(1 - beta,2)*pow(sigma,2)*
                                (Actual365Fixed().yearFraction(eval_date, tenor))))
        b = 1/(1 - beta)
        f = (pow(S0, 2 - 2*beta)/(pow(1 - beta, 2)*pow(sigma, 2)*
                                  (Actual365Fixed().yearFraction(eval_date, tenor))))

        price = N * PVBP * (S0*(stats.ncx2.cdf(d, b+2, f)) - K*(1 - stats.ncx2.cdf(f, b, d)))
        return(price)

    def sigma_CEV_Hagan(self, S0, K,sigma, beta, eval_date, expiry):
        #Set global settings
        self.eval_date()

        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        Sav = (S0+K)/2
        theta = S0 - K
        delta = ((sigma * (Sav**beta))**2)*Actual365Fixed().yearFraction(eval_date, expiry)

        term1 = sigma/(pow(Sav, 1 - beta))
        term2 = (pow(1 - beta, 2)*delta)/ (24 * pow(Sav,2))
        term3 = (1 - beta)*(2 + beta)*(pow(theta, 2)/(24 *pow(Sav,2)))
        term4 = (pow(1 - beta,2)*pow(delta,2))/((1920 *pow(Sav,4))*(7 - 54*beta + 27*pow(beta,2)))
        term5 = (pow(1 - beta,2)*pow(delta,2))/(2880 *pow(Sav,4))*(29 - 13*beta - 16*pow(beta,2))
        term6 = (1 - beta)*(delta**4)/((5760 *pow(Sav,4))*(72 - 34*beta - 9*pow(beta,2) - 7*pow(beta,3)))

        sigma_black = term1 * (1 + term2 + term3 + term4 + term5 + term6)

        return(sigma_black)


    '''Shifted CEV'''
    def sigma_shifted_CEV_Hagan(self, S0, K, sigma, delta, eta, eval_date, expiry):
        #Set global settings
        self.eval_date()
        
        #Set the discounting curve
        eonia_curve_ld = self.eonia_construct()
        
        '''NOTE: Difference between Delta and delta'''
        Sav = (S0+K)/2
        theta = S0 - K
        Delta = ((sigma * (Sav-delta)**eta)**2)*Thirty360().yearFraction(eval_date, expiry)
        gamma1 = eta/((Sav - delta))
        gamma2 = (eta *(eta - 1))/((Sav - delta)**2)
        gamma3 = (eta*(eta - 1)*(eta -2))/((Sav - delta)**3)
        gamma4 = (eta*(eta - 1)*(eta - 2)*(eta - 3))/((Sav - delta)**4)

        term1 = sigma*((Sav - delta)**eta)/Sav

        term2 =  (Delta/24) *(2 * gamma2 - gamma1**2 + 1/Sav**2)    
        term3 = (theta**2/24) *(gamma2 - 2 *gamma1**2 + 2/Sav**2)

        term4 = ((Delta**2/480)*(2*gamma4 + 4*gamma1*gamma3 - 3*gamma2**2 + 3*(gamma1**2)*gamma2 + 
                                 0.75*(gamma1**4) - 0.75/Sav**4))
        term5 =  (Delta**2/480) *((10*gamma2 - 5 *gamma1**2 + 5/(Sav**2))/(2*Sav))

        term6 = ((((theta**2)*Delta)/2880) *(6*gamma4 - 18*gamma1*gamma3 +14*gamma2**2 - 
                                           29*(gamma1**2)*gamma2 + 11*gamma1**4 - 11/Sav**4))
        term7 =  (((theta**2)*Delta)/2880) *((35*gamma2 -  40*gamma1**2 + 40/(Sav**2))/(Sav))

        term8 = (((theta**4)/1440) *(0.75*gamma4 - 6*gamma1*gamma3 - 2*gamma2**2 + 
                                17*(gamma1**2)*gamma2 - 8*gamma1**4 + 8/Sav**4))
        term9 =  ((theta**4)/1440) *((5*gamma2 -  10*gamma1**2 + 10/(Sav**2))/(Sav))

        sigma_black = term1 * (1 + term2 + term3 + term4 + term5 + term6+ term7+term8 + term9)

        return(sigma_black)

In [ ]:
'''BLACK SWAPTION PRICER'''
# Build the swaption
def payer_Black(N, S0, expiry, tenor, K, eval_date,  sigma):
    e_date = DateParser.parseFormatted(eval_date, '%Y-%m-%d')
    Settings.instance().setEvaluationDate(eval_date)
    
    # Find the schedule of payments for the underlying swap 
    schedule = Schedule(expiry,
         tenor,
         Period(Annual),
         TARGET(),
         ModifiedFollowing,
         ModifiedFollowing,
         DateGeneration.Forward,
         False)
    
    # Sum of Zero Coupon Bonds
    PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])
    
    #Define Parameters
    T = Actual365Fixed().yearFraction(eval_date, expiry)
    d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    
    #Calculate Price
    price = N*PVBP * (S0*stats.norm.cdf(d1) - K*stats.norm.cdf(d2))
    return(price)

def receiver_Black(N, S0, expiry, tenor, payments, K, sigma):
    schedule = Schedule(expiry,
         tenor,
         Period(Annual),
         TARGET(),
         ModifiedFollowing,
         ModifiedFollowing,
         DateGeneration.Forward,
         False)
    PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])
    T = Actual365Fixed().yearFraction(eval_date, expiry)
    d1 = (log(S0/K) + 0.5*(pow(sigma,2)*T))/(sigma * sqrt(T))
    d2 = d1 - sigma*sqrt(T)
          
    price = N * PVBP * (K*stats.norm.cdf(-d2) - S0*stats.norm.cdf(-d1))      
    return(price)    

In [ ]:
'''Terms obtained from Andersen & Andreasen(2000)
NOTE: This function works uniquely for beta values BELOW 1

TERMS
===============
S0 - Current Forward Swap Rate
expiry - Expiry date of the swaption
tenor - Maturity of the underlying forward swap
payments - Payment schedule of the underlying forward swap
K - Strike of the swaption
beta - Power function in the CEV Process
sigma - Volatility term'''

def payer_chi_square(N, S0, expiry, tenor, K, beta, sigma):
    schedule = Schedule(expiry,
         tenor,
         Period(Annual),
         TARGET(),
         ModifiedFollowing,
         ModifiedFollowing,
         DateGeneration.Forward,
         False)
    
    PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])]) 
    
    # Parameters for input into Chi Square CDF
    d = (pow(K,2 - 2*beta)/(pow(1 - beta,2)*pow(sigma,2)*
                            (Actual365Fixed().yearFraction(eval_date, tenor))))
    b = 1/(1 - beta)
    f = (pow(S0, 2 - 2*beta)/(pow(1 - beta, 2)*pow(sigma, 2)*
                              (Actual365Fixed().yearFraction(eval_date, tenor))))
    
    # Calculate Price
    price = N * PVBP * (S0*(1 - stats.ncx2.cdf(d, b+2, f)) - K*(stats.ncx2.cdf(f, b, d)))
    return(price)

def receiver_chi_square(N, S0, expiry, tenor, K, beta, sigma):
    schedule = Schedule(expiry,
         tenor,
         Period(Annual),
         TARGET(),
         ModifiedFollowing,
         ModifiedFollowing,
         DateGeneration.Forward,
         False)
        
    PVBP = np.sum([eonia_curve_ld.discount(date) for date in np.array(list(schedule)[1:])])
        
    d = (pow(K,2 - 2*beta)/(pow(1 - beta,2)*pow(sigma,2)*
                            (Actual365Fixed().yearFraction(eval_date, tenor))))
    b = 1/(1 - beta)
    f = (pow(S0, 2 - 2*beta)/(pow(1 - beta, 2)*pow(sigma, 2)*
                              (Actual365Fixed().yearFraction(eval_date, tenor))))
    
    price = N * PVBP * (S0*(stats.ncx2.cdf(d, b+2, f)) - K*(1 - stats.ncx2.cdf(f, b, d)))
    return(price)

In [1]:
'''HAGAN APPROXIMATION'''
'''Based on Hagan & Woodward(1998)'''
'''CEV'''
def sigma_CEV_Hagan(S0, K,sigma, beta, eval_date, expiry):
    Sav = (S0+K)/2
    theta = S0 - K
    delta = ((sigma * (Sav**beta))**2)*Actual365Fixed().yearFraction(eval_date, expiry)
    
    term1 = sigma/(pow(Sav, 1 - beta))
    term2 = (pow(1 - beta, 2)*delta)/ (24 * pow(Sav,2))
    term3 = (1 - beta)*(2 + beta)*(pow(theta, 2)/(24 *pow(Sav,2)))
    term4 = (pow(1 - beta,2)*pow(delta,2))/((1920 *pow(Sav,4))*(7 - 54*beta + 27*pow(beta,2)))
    term5 = (pow(1 - beta,2)*pow(delta,2))/(2880 *pow(Sav,4))*(29 - 13*beta - 16*pow(beta,2))
    term6 = (1 - beta)*(delta**4)/((5760 *pow(Sav,4))*(72 - 34*beta - 9*pow(beta,2) - 7*pow(beta,3)))
    
    sigma_black = term1 * (1 + term2 + term3 + term4 + term5 + term6)
    
    return(sigma_black)


'''Shifted CEV'''
def sigma_shifted_CEV_Hagan(S0, K, sigma, delta, eta, eval_date, expiry):
    '''NOTE: Difference between Delta and delta'''
    Sav = (S0+K)/2
    theta = S0 - K
    Delta = ((sigma * (Sav-delta)**eta)**2)*Thirty360().yearFraction(eval_date, expiry)
    gamma1 = eta/((Sav - delta))
    gamma2 = (eta *(eta - 1))/((Sav - delta)**2)
    gamma3 = (eta*(eta - 1)*(eta -2))/((Sav - delta)**3)
    gamma4 = (eta*(eta - 1)*(eta - 2)*(eta - 3))/((Sav - delta)**4)
    
    term1 = sigma*((Sav - delta)**eta)/Sav
    
    term2 =  (Delta/24) *(2 * gamma2 - gamma1**2 + 1/Sav**2)    
    term3 = (theta**2/24) *(gamma2 - 2 *gamma1**2 + 2/Sav**2)
    
    term4 = ((Delta**2/480)*(2*gamma4 + 4*gamma1*gamma3 - 3*gamma2**2 + 3*(gamma1**2)*gamma2 + 
                             0.75*(gamma1**4) - 0.75/Sav**4))
    term5 =  (Delta**2/480) *((10*gamma2 - 5 *gamma1**2 + 5/(Sav**2))/(2*Sav))
    
    term6 = ((((theta**2)*Delta)/2880) *(6*gamma4 - 18*gamma1*gamma3 +14*gamma2**2 - 
                                       29*(gamma1**2)*gamma2 + 11*gamma1**4 - 11/Sav**4))
    term7 =  (((theta**2)*Delta)/2880) *((35*gamma2 -  40*gamma1**2 + 40/(Sav**2))/(Sav))
    
    term8 = (((theta**4)/1440) *(0.75*gamma4 - 6*gamma1*gamma3 - 2*gamma2**2 + 
                            17*(gamma1**2)*gamma2 - 8*gamma1**4 + 8/Sav**4))
    term9 =  ((theta**4)/1440) *((5*gamma2 -  10*gamma1**2 + 10/(Sav**2))/(Sav))
    
    sigma_black = term1 * (1 + term2 + term3 + term4 + term5 + term6+ term7+term8 + term9)
    
    return(sigma_black)